In [1]:
data_root <- "/shared/0/projects/news-quotes/"

In [3]:
library("lme4")
library("margins")
library("stargazer")
library("ggeffects")

In [4]:
mydata <- read.csv(paste(data_root, "reg_data.csv", sep = ''), header = TRUE)

In [5]:
nrow(mydata)

[1] 524052

In [6]:
ncol(mydata)

[1] 249

In [7]:
# coarsen reporters into 4 groups.
mydata$reporter_eth_ethnea_broad <- as.character(mydata$reporter_eth_ethnea_broad)
mydata$reporter_eth_ethnea_broad[mydata$reporter_eth_ethnea_broad == 'EasternEuropean'] <- 'European'
mydata$reporter_eth_ethnea_broad[mydata$reporter_eth_ethnea_broad == 'WesternNorthernEuropean'] <- 'European'
mydata$reporter_eth_ethnea_broad[mydata$reporter_eth_ethnea_broad == 'SouthernEuropean'] <- 'European'
mydata$reporter_eth_ethnea_broad[mydata$reporter_eth_ethnea_broad == 'Chinese'] <- 'Asian'
mydata$reporter_eth_ethnea_broad[mydata$reporter_eth_ethnea_broad == 'Indian'] <- 'Asian'
mydata$reporter_eth_ethnea_broad[mydata$reporter_eth_ethnea_broad == 'EastAsian'] <- 'Asian'
mydata$reporter_eth_ethnea_broad[mydata$reporter_eth_ethnea_broad == 'MiddleEastern'] <- 'OtherUnknown'
mydata$reporter_eth_ethnea_broad[mydata$reporter_eth_ethnea_broad == 'African'] <- 'OtherUnknown'
mydata$reporter_eth_ethnea_broad[mydata$reporter_eth_ethnea_broad == 'unknown'] <- 'OtherUnknown'
mydata$reporter_eth_ethnea_broad <- as.factor(mydata$reporter_eth_ethnea_broad)

In [8]:
mydata <- within(mydata, author_eth_ethnea_broad <- relevel(author_eth_ethnea_broad, ref = 'English'))
mydata <- within(mydata, reporter_eth_ethnea_broad <- relevel(reporter_eth_ethnea_broad, ref = 'English'))
mydata <- within(mydata, author_gender_ethnea <- relevel(author_gender_ethnea, ref = 'M'))
mydata <- within(mydata, reporter_gender_ethnea <- relevel(reporter_gender_ethnea, ref = 'M'))

In [9]:
mydata <- within(mydata, author_pos_cate <- relevel(author_pos_cate, ref = 'last_position'))
mydata <- within(mydata, is_top_author <- relevel(is_top_author, ref = 'yes'))
mydata <- within(mydata, is_corresponding <- relevel(is_corresponding, ref = 'yes'))
mydata <- within(mydata, affiliation_cate <- relevel(affiliation_cate, ref = 'domestic'))

In [15]:
base_str <- "is_author_mentioned ~ 1 + author_gender_ethnea + reporter_eth_ethnea_broad + reporter_gender_ethnea + \
          last_name_length + last_name_prob + author_pos_cate + author_rank + is_top_author + is_corresponding + \
          affiliation_rank + affiliation_cate + num_authors + mention_year_center + gap_in_years + \
          num_words + num_mentioned_papers + FleschReadingEase + sentences_per_paragraph + type_token_ratio"

# 199 keywords
keywords <- " + Cell_biology + Genetics + Biology + Body_mass_index + Health_care + Disease + Gerontology + Population + Public_health + Medicine + Materials_science + Composite_material + Nanotechnology + Cohort_study + Social_psychology + Cohort + Psychological_intervention + Young_adult + Family_medicine + Cancer + Surgery + Randomized_controlled_trial + Placebo + Clinical_trial + Nursing + Applied_psychology + Human_factors_and_ergonomics + Injury_prevention + Suicide_prevention + Psychiatry + Occupational_safety_and_health + Intensive_care_medicine + Pediatrics + Hazard_ratio + Confidence_interval + Retrospective_cohort_study + Vaccination + Psychology + Perception + Cognition + Environmental_health + Obesity + Risk_factor + Quality_of_life + Physical_therapy + Weight_loss + Anatomy + Mental_health + Psychosocial + Anxiety + Distress + Business + Public_relations + Marketing + Immunology + Global_warming + Economics + Climatology + Climate_change + General_surgery + Endocrinology + Internal_medicine + Receptor + Inflammation + Stimulus__physiology_ + Immune_system + Meta_analysis + Sociology + Gene + Cancer_research + Breast_cancer + Cell + Diabetes_mellitus + Blood_pressure + Oncology + Gynecology + Communication + Cognitive_psychology + Adverse_effect + Clinical_endpoint + Pharmacology + Virology + Risk_assessment + Transcription_factor + Political_science + Ecology + Geography + Cross_sectional_study + Odds_ratio + Comorbidity + Environmental_engineering + Chemistry + Medical_emergency + Physics + Social_science + Ethnic_group + Labour_economics + Antibody + Geomorphology + Geophysics + Geology + Ranging + Stroke + Environmental_resource_management + Type_2_diabetes + Cardiology + Molecular_biology + Developmental_psychology + Agriculture + Signal_transduction + Optoelectronics + Psychotherapist + Affect__psychology_ + Clinical_psychology + Anesthesia + Atmospheric_sciences + In_vivo + Biochemistry + Analytical_chemistry + Neuroscience + Botany + Gene_expression + Politics + Demography + Socioeconomic_status + Mortality_rate + Virus + Optics + Condensed_matter_physics + Bioinformatics + Law + Physical_medicine_and_rehabilitation + Stem_cell + Biodiversity + Astrophysics + Astronomy + Radiology + Pathology + Proportional_hazards_model + Chemotherapy + Predation + Food_science + Artificial_intelligence + Overweight + Antibiotics + Microbiology + Zoology + Paleontology + Habitat + Public_administration + Ecosystem + Economic_growth + Organic_chemistry + Government + Autism + Transplantation + Gastroenterology + Insulin + Engineering + Computer_science + Observational_study + Heart_disease + Epidemiology + Obstetrics + Pregnancy + Fishery + Alternative_medicine + Logistic_regression + Offspring + Mood + Bacteria + Prostate_cancer + Evolutionary_biology + Phenomenon + Longitudinal_study + Genome + Mutation + Pedagogy + Dementia + Relative_risk + Microeconomics + Odds + Feeling + Oceanography + Emergency_medicine + Personality + Prospective_cohort_study + Hippocampus + Greenhouse_gas + Biomarker__medicine_ + Myocardial_infarction + Socioeconomics + Drug + Environmental_science + Epigenetics + Inorganic_chemistry + Emergency_department + Medical_prescription + Phenotype"

In [16]:
equation_bar <- as.formula(paste(base_str, keywords, " + (1|journal_title) + (1|outlet)", sep = " "))

### English (British-origin)

In [12]:
subdata = mydata[mydata$author_eth_ethnea_broad == 'English', ]

In [13]:
nrow(subdata)

[1] 234510

In [17]:
m_eng <- glmer(formula = equation_bar, data = subdata, family = "binomial", control = glmerControl(optimizer = "nloptwrap"), nAGQ = 0)

In [133]:
summary(m_eng)$coefficients["mention_year_center", ]

Estimate    Std. Error       z value      Pr(>|z|) 
-5.142146e-02  3.474336e-03 -1.480037e+01  1.456809e-49

In [14]:
# marg_eff <- marginal_effects(m_eng, variables = "mention_year_center")

In [17]:
# write.csv(marg_eff, "/shared/0/projects/news-quotes/reg_results/year/English.csv", row.names = FALSE)

In [134]:
margins_eff <- margins(m_eng, data = subdata, variables = "mention_year_center", eps = 1)

In [135]:
write.csv(summary(margins_eff), "/shared/0/projects/news-quotes/reg_results/year/margins_one_year_change/English_margins.csv", row.names = FALSE)

### Southern European

In [136]:
subdata = mydata[mydata$author_eth_ethnea_broad == 'SouthernEuropean', ]

In [137]:
nrow(subdata)

[1] 51134

In [138]:
m_rom <- glmer(formula = equation_bar, data = subdata, family = "binomial", control = glmerControl(optimizer = "nloptwrap"), nAGQ = 0)

Warning message:
“Some predictor variables are on very different scales: consider rescaling”

In [139]:
summary(m_rom)$coefficients["mention_year_center", ]

Estimate    Std. Error       z value      Pr(>|z|) 
-4.592027e-02  7.721969e-03 -5.946706e+00  2.735929e-09

In [140]:
margins_eff <- margins(m_rom, data = subdata, variables = "mention_year_center", eps = 1)

In [141]:
write.csv(summary(margins_eff), "/shared/0/projects/news-quotes/reg_results/year/margins_one_year_change/SouthernEuropean_margins.csv", row.names = FALSE)

### WesternNorthern European

In [142]:
subdata = mydata[mydata$author_eth_ethnea_broad == 'WesternNorthernEuropean', ]

In [143]:
nrow(subdata)

[1] 106331

In [144]:
m_north_euro <- glmer(formula = equation_bar, data = subdata, family = "binomial", control = glmerControl(optimizer = "nloptwrap"), nAGQ = 0)

Warning message:
“Some predictor variables are on very different scales: consider rescaling”

In [145]:
summary(m_north_euro)$coefficients["mention_year_center", ]

Estimate    Std. Error       z value      Pr(>|z|) 
-6.226799e-02  5.201254e-03 -1.197173e+01  4.997803e-33

In [146]:
margins_eff <- margins(m_north_euro, data = subdata, variables = "mention_year_center", eps = 1)

In [147]:
write.csv(summary(margins_eff), "/shared/0/projects/news-quotes/reg_results/year/margins_one_year_change/WesternNorthernEuropean_margins.csv", row.names = FALSE)

### Chinese

In [148]:
subdata = mydata[mydata$author_eth_ethnea_broad == 'Chinese', ]

In [149]:
nrow(subdata)

[1] 43039

In [150]:
m_chinese <- glmer(formula = equation_bar, data = subdata, family = "binomial", control = glmerControl(optimizer = "nloptwrap"), nAGQ = 0)

Warning message:
“Some predictor variables are on very different scales: consider rescaling”

In [151]:
summary(m_chinese)$coefficients["mention_year_center", ]

Estimate    Std. Error       z value      Pr(>|z|) 
-5.250958e-02  8.397970e-03 -6.252652e+00  4.035403e-10

In [152]:
margins_eff <- margins(m_chinese, data = subdata, variables = "mention_year_center", eps = 1)

In [153]:
write.csv(summary(margins_eff), "/shared/0/projects/news-quotes/reg_results/year/margins_one_year_change/Chinese_margins.csv", row.names = FALSE)

### Middle Eastern

In [154]:
subdata = mydata[mydata$author_eth_ethnea_broad == 'MiddleEastern', ]

In [155]:
nrow(subdata)

[1] 26082

In [156]:
m_mid <- glmer(formula = equation_bar, data = subdata, family = "binomial", control = glmerControl(optimizer = "nloptwrap"), nAGQ = 0)

Warning message:
“Some predictor variables are on very different scales: consider rescaling”

In [157]:
summary(m_mid)$coefficients["mention_year_center", ]

Estimate    Std. Error       z value      Pr(>|z|) 
-6.495030e-02  1.181179e-02 -5.498768e+00  3.824529e-08

In [158]:
margins_eff <- margins(m_mid, data = subdata, variables = "mention_year_center", eps = 1)

In [159]:
write.csv(summary(margins_eff), "/shared/0/projects/news-quotes/reg_results/year/margins_one_year_change/MiddleEast_margins.csv", row.names = FALSE)

### Eastern European

In [160]:
subdata = mydata[mydata$author_eth_ethnea_broad == 'EasternEuropean', ]

In [161]:
nrow(subdata)

[1] 17251

In [162]:
m_east_euro <- glmer(formula = equation_bar, data = subdata, family = "binomial", control = glmerControl(optimizer = "nloptwrap"), nAGQ = 0)

Warning message:
“Some predictor variables are on very different scales: consider rescaling”

In [163]:
summary(m_east_euro)$coefficients["mention_year_center", ]

Estimate    Std. Error       z value      Pr(>|z|) 
-0.0509137569  0.0140656827 -3.6197145888  0.0002949281

In [164]:
margins_eff <- margins(m_east_euro, data = subdata, variables = "mention_year_center", eps = 1)

In [165]:
write.csv(summary(margins_eff), "/shared/0/projects/news-quotes/reg_results/year/margins_one_year_change/EastEurope_margins.csv", row.names = FALSE)

### Indian

In [166]:
subdata = mydata[mydata$author_eth_ethnea_broad == 'Indian', ]

In [167]:
nrow(subdata)

[1] 21314

In [168]:
m_ind <- glmer(formula = equation_bar, data = subdata, family = "binomial", control = glmerControl(optimizer = "nloptwrap"), nAGQ = 0)

Warning message:
“Some predictor variables are on very different scales: consider rescaling”

In [169]:
summary(m_ind)$coefficients["mention_year_center", ]

Estimate    Std. Error       z value      Pr(>|z|) 
-7.521918e-02  1.348289e-02 -5.578863e+00  2.420964e-08

In [170]:
margins_eff <- margins(m_ind, data = subdata, variables = "mention_year_center", eps = 1)

In [171]:
write.csv(summary(margins_eff), "/shared/0/projects/news-quotes/reg_results/year/margins_one_year_change/Indian_margins.csv", row.names = FALSE)

### non-Chinese East Asian

In [172]:
subdata = mydata[mydata$author_eth_ethnea_broad == 'EastAsian', ]

In [173]:
nrow(subdata)

[1] 19068

In [174]:
summary(subdata$is_top_author)

yes    no 
    0 19068

In [175]:
# removed top author status
base_str <- "is_author_mentioned ~ 1 + author_gender_ethnea + reporter_eth_ethnea_broad + reporter_gender_ethnea + \
          last_name_length + last_name_prob + author_pos_cate + author_rank + is_corresponding + \
          affiliation_rank + affiliation_cate + num_authors + mention_year_center + gap_in_years + \
          num_words + num_mentioned_papers + FleschReadingEase + sentences_per_paragraph + type_token_ratio"
equation_bar <- as.formula(paste(base_str, keywords, " + (1|journal_title) + (1|outlet)", sep = " "))

In [176]:
m_asia <- glmer(formula = equation_bar, data = subdata, family = "binomial", control = glmerControl(optimizer = "nloptwrap"), nAGQ = 0)

Warning message:
“Some predictor variables are on very different scales: consider rescaling”

In [177]:
summary(m_asia)$coefficients["mention_year_center", ]

Estimate   Std. Error      z value     Pr(>|z|) 
-0.034830880  0.013433935 -2.592753334  0.009521103

In [178]:
margins_eff <- margins(m_asia, data = subdata, variables = "mention_year_center", eps = 1)

In [179]:
write.csv(summary(margins_eff), "/shared/0/projects/news-quotes/reg_results/year/margins_one_year_change/Asia_margins.csv", row.names = FALSE)

### African (insufficient data for fitting a model)

In [112]:
subdata = mydata[mydata$author_eth_ethnea_broad == 'African', ]

In [113]:
nrow(subdata)

[1] 2774

In [114]:
m_africa <- glmer(formula = equation_bar, data = subdata, family = "binomial", control = glmerControl(optimizer = "nloptwrap"), nAGQ = 0)

ERROR: Error in `contrasts<-`(`*tmp*`, value = contr.funs[1 + isOF[nn]]): contrasts can be applied only to factors with 2 or more levels


## Gender

In [180]:
base_str <- "is_author_mentioned ~ 1 + author_eth_ethnea_broad + reporter_eth_ethnea_broad + reporter_gender_ethnea + \
          last_name_length + last_name_prob + author_pos_cate + author_rank + is_top_author + is_corresponding + \
          affiliation_rank + affiliation_cate + num_authors + mention_year_center + gap_in_years + \
          num_words + num_mentioned_papers + FleschReadingEase + sentences_per_paragraph + type_token_ratio"

In [181]:
equation_bar <- as.formula(paste(base_str, keywords, " + (1|journal_title) + (1|outlet)", sep = " "))

### Male

In [182]:
subdata = mydata[mydata$author_gender_ethnea == 'M', ]

In [183]:
nrow(subdata)

[1] 294158

In [184]:
m_male <- glmer(formula = equation_bar, data = subdata, family = "binomial", control = glmerControl(optimizer = "nloptwrap"), nAGQ = 0)

Warning message:
“Some predictor variables are on very different scales: consider rescaling”

In [185]:
summary(m_male)$coefficients["mention_year_center", ]

Estimate    Std. Error       z value      Pr(>|z|) 
-6.190934e-02  3.037717e-03 -2.038022e+01  2.505177e-92

In [186]:
margins_eff <- margins(m_male, data = subdata, variables = "mention_year_center", eps = 1)

In [187]:
write.csv(summary(margins_eff), "/shared/0/projects/news-quotes/reg_results/year/margins_one_year_change/M_margins.csv", row.names = FALSE)

### Female

In [188]:
subdata = mydata[mydata$author_gender_ethnea == 'F', ]

In [189]:
nrow(subdata)

[1] 142897

In [190]:
m_female <- glmer(formula = equation_bar, data = subdata, family = "binomial", control = glmerControl(optimizer = "nloptwrap"), nAGQ = 0)

Warning message:
“Some predictor variables are on very different scales: consider rescaling”

In [191]:
summary(m_female)$coefficients["mention_year_center", ]

Estimate    Std. Error       z value      Pr(>|z|) 
-2.870659e-02  4.693657e-03 -6.116039e+00  9.592979e-10

In [192]:
margins_eff <- margins(m_female, data = subdata, variables = "mention_year_center", eps = 1)

In [193]:
write.csv(summary(margins_eff), "/shared/0/projects/news-quotes/reg_results/year/margins_one_year_change/F_margins.csv", row.names = FALSE)